# 자동차 이미지들을 학습 및 테스트합니다.

훈련 데이터는 각 480개씩 총 1920개로 구성되어있고,
시험 데이터는 각 160개씩 총 480개로 구성되어있습니다.

차종은 세단, 쿠페, SUV, 픽업트럭으로 네가지를 구분하였고 종류별 숫자 코드는 다음과 같습니다.
0 = Sedan, 1 = Coupe, 2 = SUV, 3 = Pickup-Truck

In [1]:
import os, random
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image

Instructions for updating:
non-resource variables are not supported in the long term


    Namin Neural Network
    ----------
    Input Layer: 100*100*3(Input Size)
    
    1st Layer: Conv1(3EA, 3*3*3, Strides=3, Padding=1),
               ReLU1(34*34*3),
               Pool1(2*2, Strides=1),
               Bn1(33*33*3)
    2nd Layer: Conv2(6EA, 6*6*3, Strides=1, Padding=VALID),
               ReLU2(28*28*6),
               Pool2(2*2, Strides=2),
               Bn2(14*14*6)
    3rd Layer: Conv3(9EA, 3*3*6, Strides=3, Padding=2),
               ReLU3(6*6*9),
               Pool3(2*2, Strides=2),
               Bn3(3*3*9)
    4th Layer: Flatten,
               Affine(W=3*3*9, B=9)
                
    Output Layer: Softmax Cross Entropy(Sedan, Coupe, SUV, Pickup-Truck)

In [2]:
# Load to Sedan, Coupe, SUV, PickupTruck Images
trainlist, testlist = [], []
with open('train.txt') as f:
    for line in f:
        tmp = line.strip().split()
        trainlist.append([tmp[0], tmp[1]])
        
with open('test.txt') as f:
    for line in f:
        tmp = line.strip().split()
        testlist.append([tmp[0], tmp[1]])

In [3]:
# Image Preprocessing
IMG_H = 100
IMG_W = 100
IMG_C = 3

def readimg(path):
    img = plt.imread(path)
    return img

def batch(path, batch_size):
    img, label, paths = [], [], []
    for i in range(batch_size):
        img.append(readimg(path[0][0]))
        label.append(int(path[0][1]))
        path.append(path.pop(0))
        
    return img, label

In [42]:
# Neural Network
num_class = 4 # Sedan, Coupe, SUV, PickupTruck

with tf.Graph().as_default() as g:
    X = tf.placeholder(tf.float32, [None, IMG_H, IMG_W, IMG_C]) # Input Layer, X = tf.placeholder(tf.float32, [None, IMG_H, IMG_W, IMG_C])
    Y = tf.placeholder(tf.int32, [None]) # Y = tf.placeholder(tf.int32, [None])
    
    with tf.variable_scope('CNN'):
        # 1st Layer(Conv1 - relu1 - maxpool1 - bn1) = 33*33*3
        conv1 = tf.layers.conv2d(X, 3, 3, (3, 3), padding='SAME', activation=tf.nn.relu)
        pool1 = tf.layers.max_pooling2d(conv1, 2, (1, 1), padding='VALID')
        bn1 = tf.compat.v1.layers.batch_normalization(pool1, training=True)
        # 2nd Layer(Conv2 - relu2 - maxpool2 - bn2) = 14*14*6
        conv2 = tf.layers.conv2d(bn1, 6, 6, (1, 1), padding='VALID', activation=tf.nn.relu)
        pool2 = tf.layers.max_pooling2d(conv2, 2, (2, 2), padding='VALID')
        bn2 = tf.compat.v1.layers.batch_normalization(pool2, training=True)
        # 3rd Layer(Conv3 - relu3 - maxpool3 - bn3) = 3*3*9
        conv3 = tf.layers.conv2d(bn2, 9, 3, (3, 3), padding='SAME', activation=tf.nn.relu)
        pool3 = tf.layers.max_pooling2d(conv3, 2, (2, 2), padding='VALID')
        bn3 = tf.compat.v1.layers.batch_normalization(pool3, training=True)
        # Fully Connected Layer(Affine)
        affine1 = tf.layers.flatten(bn3)
        # Output Layer
        output = tf.layers.dense(affine1, num_class)
        
    # Softmax with Loss
    with tf.variable_scope('Loss'):
        Loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels= Y, logits=output))
    
    # Training with Adam    
    train_step = tf.train.AdamOptimizer(0.005).minimize(Loss)
    saver = tf.train.Saver()
    
    tf.summary.scalar('Epoch-Loss', Loss)
    merged = tf.summary.merge_all()

In [43]:
# Size
np.sum([np.product(var.shape) for var in g.get_collection('trainable_variables')]).value

1417

In [46]:
# Setting Batch with Training
batch_size = 32
epoch = 1000

with tf.Session(graph=g) as sess:
    sess.run(tf.global_variables_initializer())
    summary_writer = tf.summary.FileWriter('tf_board', sess.graph)
    for i in range(epoch):
        batch_data, batch_label = batch(trainlist, batch_size)     
        _, loss, summary = sess.run([train_step, Loss, merged], feed_dict = {X: batch_data, Y: batch_label})
        print("Epoch:",i+1,"Loss:",loss)
        if i % 10 == 0:
            summary_writer.add_summary(summary, i)
            saver.save(sess, 'logs/model.ckpt', global_step = i+1)
        elif i+1 == epoch:
            summary_writer.add_summary(summary, i)
            saver.save(sess, 'logs/model.ckpt', global_step = i+1)

Epoch: 1 Loss: 1.631593
Epoch: 2 Loss: 1.4582636
Epoch: 3 Loss: 1.4053605
Epoch: 4 Loss: 1.4826719
Epoch: 5 Loss: 1.2624273
Epoch: 6 Loss: 2.9279804
Epoch: 7 Loss: 2.7565608
Epoch: 8 Loss: 2.7209485
Epoch: 9 Loss: 2.775003
Epoch: 10 Loss: 2.4362888
Epoch: 11 Loss: 1.7852826
Epoch: 12 Loss: 1.7954918
Epoch: 13 Loss: 1.7305063
Epoch: 14 Loss: 1.5103219
Epoch: 15 Loss: 1.176048
Epoch: 16 Loss: 1.0825949
Epoch: 17 Loss: 1.1761502
Epoch: 18 Loss: 1.110229
Epoch: 19 Loss: 0.7596741
Epoch: 20 Loss: 0.81031716
Epoch: 21 Loss: 2.6192067
Epoch: 22 Loss: 2.2232757
Epoch: 23 Loss: 2.9230404
Epoch: 24 Loss: 2.4288945
Epoch: 25 Loss: 2.694233
Epoch: 26 Loss: 2.3608837
Epoch: 27 Loss: 2.2462497
Epoch: 28 Loss: 2.2896233
Epoch: 29 Loss: 2.3738642
Epoch: 30 Loss: 1.8606273
Epoch: 31 Loss: 1.8561676
Epoch: 32 Loss: 1.8118513
Epoch: 33 Loss: 1.5877088
Epoch: 34 Loss: 1.5532078
Epoch: 35 Loss: 1.6096509
Epoch: 36 Loss: 2.7028482
Epoch: 37 Loss: 2.554895
Epoch: 38 Loss: 2.6863909
Epoch: 39 Loss: 2.3784971


Epoch: 312 Loss: 1.4865507
Epoch: 313 Loss: 1.5235819
Epoch: 314 Loss: 1.4288926
Epoch: 315 Loss: 1.3336452
Epoch: 316 Loss: 1.3632426
Epoch: 317 Loss: 1.3408657
Epoch: 318 Loss: 1.2621431
Epoch: 319 Loss: 1.1549621
Epoch: 320 Loss: 1.19023
Epoch: 321 Loss: 1.4960024
Epoch: 322 Loss: 1.4773941
Epoch: 323 Loss: 1.6258179
Epoch: 324 Loss: 1.477725
Epoch: 325 Loss: 1.617972
Epoch: 326 Loss: 1.547442
Epoch: 327 Loss: 1.5179245
Epoch: 328 Loss: 1.5432594
Epoch: 329 Loss: 1.6203239
Epoch: 330 Loss: 1.4786117
Epoch: 331 Loss: 1.5365031
Epoch: 332 Loss: 1.5083761
Epoch: 333 Loss: 1.5354925
Epoch: 334 Loss: 1.487242
Epoch: 335 Loss: 1.4093521
Epoch: 336 Loss: 1.506464
Epoch: 337 Loss: 1.4201272
Epoch: 338 Loss: 1.4515096
Epoch: 339 Loss: 1.4628547
Epoch: 340 Loss: 1.425905
Epoch: 341 Loss: 1.3792734
Epoch: 342 Loss: 1.5262915
Epoch: 343 Loss: 1.3636243
Epoch: 344 Loss: 1.460383
Epoch: 345 Loss: 1.4443798
Epoch: 346 Loss: 1.370183
Epoch: 347 Loss: 1.4031246
Epoch: 348 Loss: 1.3559666
Epoch: 349 

Epoch: 617 Loss: 1.3089665
Epoch: 618 Loss: 1.2669182
Epoch: 619 Loss: 1.1559304
Epoch: 620 Loss: 1.1743021
Epoch: 621 Loss: 1.4338845
Epoch: 622 Loss: 1.4142257
Epoch: 623 Loss: 1.5142689
Epoch: 624 Loss: 1.4429355
Epoch: 625 Loss: 1.5205086
Epoch: 626 Loss: 1.4758408
Epoch: 627 Loss: 1.4297085
Epoch: 628 Loss: 1.4583211
Epoch: 629 Loss: 1.5933996
Epoch: 630 Loss: 1.4024162
Epoch: 631 Loss: 1.4799466
Epoch: 632 Loss: 1.4624243
Epoch: 633 Loss: 1.4707102
Epoch: 634 Loss: 1.4603125
Epoch: 635 Loss: 1.3683522
Epoch: 636 Loss: 1.4076896
Epoch: 637 Loss: 1.3595269
Epoch: 638 Loss: 1.391895
Epoch: 639 Loss: 1.3799579
Epoch: 640 Loss: 1.3388453
Epoch: 641 Loss: 1.3109713
Epoch: 642 Loss: 1.481111
Epoch: 643 Loss: 1.285934
Epoch: 644 Loss: 1.4377489
Epoch: 645 Loss: 1.3673991
Epoch: 646 Loss: 1.3118596
Epoch: 647 Loss: 1.315045
Epoch: 648 Loss: 1.3126951
Epoch: 649 Loss: 1.3897561
Epoch: 650 Loss: 1.2907052
Epoch: 651 Loss: 1.4870887
Epoch: 652 Loss: 1.5472314
Epoch: 653 Loss: 1.4394007
Epoch

Epoch: 922 Loss: 1.4327697
Epoch: 923 Loss: 1.3659828
Epoch: 924 Loss: 1.4128721
Epoch: 925 Loss: 1.4358959
Epoch: 926 Loss: 1.4282188
Epoch: 927 Loss: 1.3597469
Epoch: 928 Loss: 1.3730984
Epoch: 929 Loss: 1.3866339
Epoch: 930 Loss: 1.3068924
Epoch: 931 Loss: 1.373078
Epoch: 932 Loss: 1.4031185
Epoch: 933 Loss: 1.3418512
Epoch: 934 Loss: 1.3035254
Epoch: 935 Loss: 1.2633023
Epoch: 936 Loss: 1.4923573
Epoch: 937 Loss: 1.4925152
Epoch: 938 Loss: 1.4510636
Epoch: 939 Loss: 1.4953983
Epoch: 940 Loss: 1.3991747
Epoch: 941 Loss: 1.4226688
Epoch: 942 Loss: 1.4953146
Epoch: 943 Loss: 1.3507348
Epoch: 944 Loss: 1.4705598
Epoch: 945 Loss: 1.4575803
Epoch: 946 Loss: 1.3596855
Epoch: 947 Loss: 1.3847039
Epoch: 948 Loss: 1.3348219
Epoch: 949 Loss: 1.395719
Epoch: 950 Loss: 1.292139
Epoch: 951 Loss: 1.6222067
Epoch: 952 Loss: 1.6820779
Epoch: 953 Loss: 1.512229
Epoch: 954 Loss: 1.5940727
Epoch: 955 Loss: 1.4537311
Epoch: 956 Loss: 1.5360022
Epoch: 957 Loss: 1.4917037
Epoch: 958 Loss: 1.5057709
Epoch

In [47]:
# Print an Accuracy
acc = 0

with tf.Session(graph=g) as sess:
    sess.run(tf.global_variables_initializer())
    checkpoint = tf.train.latest_checkpoint('logs')
    if checkpoint:
        saver.restore(sess, checkpoint)
    for i in range(len(testlist)):
        batch_data, batch_label = batch(testlist, 1)
        logit = sess.run(output, feed_dict = {X:batch_data})
        if np.argmax(logit[0]) == batch_label[0]:
            acc += 1
        else:
            print(logit[0], batch_label[0])
            
    print("Accuracy:", acc/len(testlist))

INFO:tensorflow:Restoring parameters from logs\model.ckpt-1000
[ 0.03118621 -0.7509834  -0.3411302   0.15451182] 0
[-0.29599497 -0.00972899  0.69436216 -0.34930193] 0
[ 0.20180067  0.685225    0.6394979  -0.24031797] 0
[ 0.3289079   0.87433904  0.48748899 -0.33216542] 0
[ 0.11306528  0.06574048 -0.15619108  0.61677486] 0
[ 0.12965593 -0.67751515 -0.2887019   0.49423605] 0
[-0.29129517 -0.87265456 -0.6146739   0.27174616] 0
[-0.31972718  0.1364482   0.00375311 -0.13819927] 0
[ 0.0067321   0.32318392 -0.36040044 -0.51187325] 0
[ 0.04719917  0.14003897 -0.19405986 -0.14264196] 0
[ 0.30392346  0.63925725  0.3851918  -0.41005743] 0
[ 0.19096169  0.6999493   0.3182152  -0.38203782] 0
[0.33381504 0.6921789  0.47539198 0.67109245] 0
[ 0.11297365  0.717613    0.43210894 -0.33377308] 0
[ 0.11597645  0.7560584   0.54772615 -0.16456857] 0
[ 0.45049363  1.091221    0.18170224 -0.7843885 ] 0
[-0.42932796 -0.8593974  -0.5069361  -0.3514501 ] 0
[ 0.0151442  -0.34857088 -0.51285434  0.20732044] 0
[-0.1

[ 0.3856552   0.5681018   0.6409953  -0.32014582] 1
[ 0.44908938 -0.33811733 -0.20125535  0.17717104] 1
[ 0.31505796 -0.0890376   0.15105163 -0.01600879] 1
[-0.4032459  -1.3947339  -0.33602485  0.52493703] 1
[ 0.33543706 -0.25665864 -0.12358163 -0.5863059 ] 1
[ 0.11301622  0.59806585  0.8039702  -0.0832848 ] 1
[ 0.32072717 -0.0213178   0.37491727  0.8169246 ] 1
[-0.2421089  -0.5553005  -0.55685484 -0.48717433] 1
[ 0.01600491 -0.6271948  -0.354168    0.22078146] 1
[0.05194828 0.31597835 0.6708177  0.2545954 ] 1
[ 0.19741166  0.46816036  0.5666187  -0.6526624 ] 1
[-0.12256848 -0.16565062  0.13269629  0.00279833] 1
[ 0.07931025 -0.31539422 -0.36157972 -0.3178647 ] 1
[-0.18391307 -0.5814933  -0.23616894  0.68584853] 1
[-0.0898267  -0.02919852  0.01486174  0.8555411 ] 1
[-0.22691594 -0.82083476 -0.53681546  0.2919857 ] 1
[-0.26882705 -0.8100487  -0.3004893   0.19256686] 1
[ 0.30403033  0.13402796 -0.09068889  0.33954734] 1
[ 0.1148923  -0.6086484   0.09132384 -0.18497354] 1
[-0.27105966 -0.

[ 0.08276757  0.40322903  0.04915206 -0.51972246] 2
[-0.90814924 -0.9806684  -0.08757046  0.13031684] 2
[ 0.23662598 -0.2917684  -0.36121696 -0.8179521 ] 2
[ 0.06787337 -0.7849353   0.27668476  0.62794554] 2
[-0.01147721  0.01033514 -0.54999596 -0.70806   ] 2
[-0.22332428 -0.71679205 -0.34136587  0.45304012] 2
[ 0.2132685  -0.68641853  0.02905499 -0.5314749 ] 2
[-0.12626365 -0.56962913 -0.15532036  0.2058293 ] 2
[ 0.87666094  0.8593758  -0.3983562  -0.47931367] 2
[-0.36470813 -0.340728   -0.4178657  -0.59267765] 3
[-0.05232818 -0.8578982  -0.9302488  -0.35284507] 3
[ 0.40465346 -0.31241325 -0.15193711  0.34487772] 3
[0.5446491  0.11296168 0.8495943  0.46318066] 3
[-0.33643493  0.6204553  -0.27358547 -0.14254066] 3
[-0.14221889  0.5271014  -0.02905449  0.13252376] 3
[ 0.6975113  -0.05748967  0.14385308  0.11003192] 3
[ 0.19550796  0.5721322   0.5123073  -0.18507224] 3
[ 0.1706574  -0.00851092  0.62260944  0.40801603] 3
[ 0.07551252  0.6190822   0.47617114 -0.59123325] 3
[ 0.22825173 -0.